In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import re
import json

from Tools import Tools4DataExtraction as T4DE

In [2]:
D4NN= T4DE.Get_Data4NN()
D4NN= D4NN.iloc[:-1]

Memoria usada antes: 627.18 MB
Memoria usada después: 121.87 MB
Reducción: 80.6%


In [3]:
D4NN.columns

Index(['FECHA_CORRIDA', 'HORA_SALIDA_ORIGEN_CORRIDA', 'TIPO_PASAJERO',
       'PAGO_METODO', 'PORCENT_PROMO', 'BOLETOS_VEND', 'AÑO', 'DIF_TARIF',
       'CLASE_SERVICIO', 'ORIGEN', 'DESTINO', 'MES', 'KMS_TRAMO',
       'OCUPACION_TRAMO', 'CAPACIDAD_ASIENTOS_TRAMO', 'HORAS_ANTICIPACION',
       'DISPONIBILIDAD_TRAMO', 'NOMBRE_DIA_CORRIDA', 'VENTA'],
      dtype='object')

In [4]:
def HourCorrection(df):
    D4NN=df.copy()
    # 1. Convertir a string (por si tiene tipo 'category' u 'object')
    D4NN['HORA_SALIDA_ORIGEN_CORRIDA'] = D4NN['HORA_SALIDA_ORIGEN_CORRIDA'].astype(str)
    
    # 2. Limpiar repeticiones dentro del texto, como "11:00:0011:00:00"
    # Explicación: busca repeticiones consecutivas y se queda solo con una
    D4NN['HORA_SALIDA_ORIGEN_CORRIDA'] = (
        D4NN['HORA_SALIDA_ORIGEN_CORRIDA']
        .apply(lambda x: re.findall(r'\d{2}:\d{2}:\d{2}', x)[0] if re.findall(r'\d{2}:\d{2}:\d{2}', x) else None)
    )
    
    # 3. Ahora convertir a datetime
    D4NN['HORA_SALIDA_ORIGEN_CORRIDA'] = pd.to_datetime(
        D4NN['HORA_SALIDA_ORIGEN_CORRIDA'], 
        format='%H:%M:%S', 
        errors='coerce'
    )
    
    D4NN['HoraSalidaOrigenCorrida'] = (D4NN['HORA_SALIDA_ORIGEN_CORRIDA'].dt.hour.astype(str))
    return D4NN


In [5]:
def GetTarget(df):
    # Definir la variable de agrupación
    AGRUPACION_VAR = 'FECHA_CORRIDA'
    #  Inicializar el DataFrame final agregado
    df_agregado = df.groupby(AGRUPACION_VAR)['BOLETOS_VEND'].sum().reset_index()
    df_agregado = df_agregado.rename(columns={'BOLETOS_VEND': 'VENTABOL_DIARIA'})
    return df_agregado

In [6]:
def Get_proportions(D4NN,df):
    AGRUPACION_VAR = 'FECHA_CORRIDA'
    #  Definir las variables categóricas a transformar
    CAT_VARS = ['HoraSalidaOrigenCorrida','PAGO_METODO', 'TIPO_PASAJERO', 'CLASE_SERVICIO']
    df_agregado= df.copy()
    for col in CAT_VARS:
        # a. Crear conteos por fecha y categoría
        df_counts = D4NN.groupby([AGRUPACION_VAR, col]).size().reset_index(name=f'CONTEO_{col}')
        
        # b. Pivotear la tabla para tener las categorías como columnas
        df_proportions = df_counts.pivot_table(
            index=AGRUPACION_VAR, 
            columns=col, 
            values=f'CONTEO_{col}', 
            fill_value=0
        )
    
        # c. Calcular proporciones
        df_proportions = df_proportions.div(df_proportions.sum(axis=1), axis=0)
    
        # d. Renombrar columnas (por claridad)
        df_proportions.columns = [f'PROP_{col}_{cat}' for cat in df_proportions.columns]
    
        # e. Hacer merge con df_agregado (por índice o por columna)
        df_agregado = df_agregado.merge(
            df_proportions, 
            how='left', 
            left_on=AGRUPACION_VAR, 
            right_index=True
        )

    return df_agregado
        

In [7]:
def Get_means(D4NN, df):
    AGRUPACION_VAR = 'FECHA_CORRIDA'
    
    Col_Num_Vars = [
        'KMS_TRAMO', 'CAPACIDAD_ASIENTOS_TRAMO', 'OCUPACION_TRAMO', 'DISPONIBILIDAD_TRAMO',
        'PORCENT_PROMO', 'HORAS_ANTICIPACION', 'DIF_TARIF', 'VENTA']
    
    # 1. Calcular los promedios por fecha
    df_means = D4NN.groupby(AGRUPACION_VAR)[Col_Num_Vars].mean().reset_index()
    
    # 2. Hacer merge con el DataFrame original
    df_agregado = df.copy()
    df_agregado = df_agregado.merge(
        df_means,
        how='left',
        on=AGRUPACION_VAR
    )
    
    return df_agregado

In [8]:
def Get_carac(Df):
    df=Df.copy()
    df['Anio']=pd.to_datetime(df['FECHA_CORRIDA']).dt.year
    df['DiaSemana']=pd.to_datetime(df['FECHA_CORRIDA']).dt.dayofweek
    df['Mes']=pd.to_datetime(df['FECHA_CORRIDA']).dt.month
    return df

In [9]:
def Get_goodCarac(Df,BC_json):
    df=Df.copy()
    df['Buen_Dia'] = df['DiaSemana'].isin(BC_json["DiaBueno"]).astype(int)
    df['Buen_Mes'] = df['Mes'].isin(BC_json["MesBueno"]).astype(int)
    return df

In [10]:
D4NN= HourCorrection(D4NN)

In [11]:
D4NN['HORA_SALIDA_ORIGEN_CORRIDA']

0        1900-01-01 08:20:00
1        1900-01-01 11:00:00
2        1900-01-01 11:00:00
3        1900-01-01 11:00:00
4        1900-01-01 11:00:00
                 ...        
394262   1900-01-01 11:00:01
394263   1900-01-01 11:00:01
394264   1900-01-01 11:00:01
394265   1900-01-01 23:59:01
394266   1900-01-01 23:59:01
Name: HORA_SALIDA_ORIGEN_CORRIDA, Length: 394267, dtype: datetime64[ns]

In [12]:
D4NN['HoraSalidaOrigenCorrida']

0          8
1         11
2         11
3         11
4         11
          ..
394262    11
394263    11
394264    11
394265    23
394266    23
Name: HoraSalidaOrigenCorrida, Length: 394267, dtype: object

In [13]:
df_agregado= GetTarget(D4NN.copy())

In [14]:
df_agregado

,FECHA_CORRIDA,VENTABOL_DIARIA
0,2023-01-01,157.0
1,2023-01-02,211.0
2,2023-01-03,448.0
3,2023-01-04,426.0
4,2023-01-05,273.0
...,...,...
1029,2025-10-26,206.0
1030,2025-10-27,163.0
1031,2025-10-28,76.0
1032,2025-10-29,38.0


In [15]:
df_agregado= Get_proportions(D4NN.copy(),df_agregado)

In [16]:
df_agregado

,FECHA_CORRIDA,VENTABOL_DIARIA,PROP_HoraSalidaOrigenCorrida_0,PROP_HoraSalidaOrigenCorrida_1,PROP_HoraSalidaOrigenCorrida_10,PROP_HoraSalidaOrigenCorrida_11,PROP_HoraSalidaOrigenCorrida_12,PROP_HoraSalidaOrigenCorrida_13,PROP_HoraSalidaOrigenCorrida_14,PROP_HoraSalidaOrigenCorrida_15,...,PROP_TIPO_PASAJERO_OD,PROP_TIPO_PASAJERO_PE,PROP_TIPO_PASAJERO_PF,PROP_CLASE_SERVICIO_DE LUJO,PROP_CLASE_SERVICIO_DOS PISOS ETN,PROP_CLASE_SERVICIO_DOS PISOS TURISTAR,PROP_CLASE_SERVICIO_DTM DOS PISOS,PROP_CLASE_SERVICIO_TURISTAR LUJO,PROP_CLASE_SERVICIO_UN PISO DE LUJO,PROP_CLASE_SERVICIO_UN PISO TURISTAR LUJO
0,2023-01-01,157.0,0.000000,0.000000,0.0,0.000000,0.000000,0.169492,0.000000,0.039548,...,0.000000,0.045198,0.0,0.706215,0.169492,0.000000,0.0,0.124294,0.0,0.0
1,2023-01-02,211.0,0.094118,0.113725,0.0,0.007843,0.133333,0.039216,0.113725,0.035294,...,0.007843,0.015686,0.0,0.392157,0.607843,0.000000,0.0,0.000000,0.0,0.0
2,2023-01-03,448.0,0.000000,0.053131,0.0,0.011385,0.000000,0.060721,0.041746,0.227704,...,0.001898,0.066414,0.0,0.092979,0.907021,0.000000,0.0,0.000000,0.0,0.0
3,2023-01-04,426.0,0.000000,0.075435,0.0,0.052224,0.000000,0.029014,0.069632,0.054159,...,0.000000,0.029014,0.0,0.009671,0.990329,0.000000,0.0,0.000000,0.0,0.0
4,2023-01-05,273.0,0.000000,0.064725,0.0,0.051780,0.038835,0.000000,0.100324,0.058252,...,0.003236,0.035599,0.0,0.000000,0.980583,0.000000,0.0,0.019417,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,2025-10-26,206.0,0.000000,0.000000,0.0,0.264957,0.000000,0.000000,0.000000,0.000000,...,0.004274,0.004274,0.0,0.000000,0.923077,0.076923,0.0,0.000000,0.0,0.0
1030,2025-10-27,163.0,0.000000,0.000000,0.0,0.251282,0.000000,0.000000,0.000000,0.000000,...,0.030769,0.000000,0.0,0.000000,0.887179,0.112821,0.0,0.000000,0.0,0.0
1031,2025-10-28,76.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.000000,0.0,0.0
1032,2025-10-29,38.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.840909,0.159091,0.0,0.000000,0.0,0.0


In [17]:
df_agregado= Get_means(D4NN.copy(),df_agregado)

In [18]:
df_agregado

,FECHA_CORRIDA,VENTABOL_DIARIA,PROP_HoraSalidaOrigenCorrida_0,PROP_HoraSalidaOrigenCorrida_1,PROP_HoraSalidaOrigenCorrida_10,PROP_HoraSalidaOrigenCorrida_11,PROP_HoraSalidaOrigenCorrida_12,PROP_HoraSalidaOrigenCorrida_13,PROP_HoraSalidaOrigenCorrida_14,PROP_HoraSalidaOrigenCorrida_15,...,PROP_CLASE_SERVICIO_UN PISO DE LUJO,PROP_CLASE_SERVICIO_UN PISO TURISTAR LUJO,KMS_TRAMO,CAPACIDAD_ASIENTOS_TRAMO,OCUPACION_TRAMO,DISPONIBILIDAD_TRAMO,PORCENT_PROMO,HORAS_ANTICIPACION,DIF_TARIF,VENTA
0,2023-01-01,157.0,0.000000,0.000000,0.0,0.000000,0.000000,0.169492,0.000000,0.039548,...,0.0,0.0,520.0,28.898305,23.282486,4.525424,10.465117,4.451977,0.544464,714.251465
1,2023-01-02,211.0,0.094118,0.113725,0.0,0.007843,0.133333,0.039216,0.113725,0.035294,...,0.0,0.0,520.0,33.027451,22.627451,6.690196,10.275229,12.298039,0.486919,696.876343
2,2023-01-03,448.0,0.000000,0.053131,0.0,0.011385,0.000000,0.060721,0.041746,0.227704,...,0.0,0.0,520.0,33.210626,24.597723,4.969639,11.102942,16.571157,0.508980,697.675415
3,2023-01-04,426.0,0.000000,0.075435,0.0,0.052224,0.000000,0.029014,0.069632,0.054159,...,0.0,0.0,520.0,35.000000,24.087040,6.959381,10.136987,14.059961,0.470546,665.716064
4,2023-01-05,273.0,0.000000,0.064725,0.0,0.051780,0.038835,0.000000,0.100324,0.058252,...,0.0,0.0,520.0,34.844660,15.161812,17.572816,10.434783,13.779935,0.533888,670.463806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,2025-10-26,206.0,0.000000,0.000000,0.0,0.264957,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,520.0,35.739316,22.094017,5.794872,3.445596,197.615385,0.663336,937.223816
1030,2025-10-27,163.0,0.000000,0.000000,0.0,0.251282,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,520.0,35.887179,20.369230,6.764103,3.266129,220.733333,0.624237,869.557068
1031,2025-10-28,76.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,520.0,36.000000,23.551723,12.448276,5.000000,6.586207,0.732618,886.694031
1032,2025-10-29,38.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,520.0,35.022727,28.590910,3.863636,4.677419,51.613636,0.555750,917.601135


In [19]:
df_agregado = Get_carac(df_agregado)

In [20]:
df_agregado

,FECHA_CORRIDA,VENTABOL_DIARIA,PROP_HoraSalidaOrigenCorrida_0,PROP_HoraSalidaOrigenCorrida_1,PROP_HoraSalidaOrigenCorrida_10,PROP_HoraSalidaOrigenCorrida_11,PROP_HoraSalidaOrigenCorrida_12,PROP_HoraSalidaOrigenCorrida_13,PROP_HoraSalidaOrigenCorrida_14,PROP_HoraSalidaOrigenCorrida_15,...,CAPACIDAD_ASIENTOS_TRAMO,OCUPACION_TRAMO,DISPONIBILIDAD_TRAMO,PORCENT_PROMO,HORAS_ANTICIPACION,DIF_TARIF,VENTA,Anio,DiaSemana,Mes
0,2023-01-01,157.0,0.000000,0.000000,0.0,0.000000,0.000000,0.169492,0.000000,0.039548,...,28.898305,23.282486,4.525424,10.465117,4.451977,0.544464,714.251465,2023,6,1
1,2023-01-02,211.0,0.094118,0.113725,0.0,0.007843,0.133333,0.039216,0.113725,0.035294,...,33.027451,22.627451,6.690196,10.275229,12.298039,0.486919,696.876343,2023,0,1
2,2023-01-03,448.0,0.000000,0.053131,0.0,0.011385,0.000000,0.060721,0.041746,0.227704,...,33.210626,24.597723,4.969639,11.102942,16.571157,0.508980,697.675415,2023,1,1
3,2023-01-04,426.0,0.000000,0.075435,0.0,0.052224,0.000000,0.029014,0.069632,0.054159,...,35.000000,24.087040,6.959381,10.136987,14.059961,0.470546,665.716064,2023,2,1
4,2023-01-05,273.0,0.000000,0.064725,0.0,0.051780,0.038835,0.000000,0.100324,0.058252,...,34.844660,15.161812,17.572816,10.434783,13.779935,0.533888,670.463806,2023,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,2025-10-26,206.0,0.000000,0.000000,0.0,0.264957,0.000000,0.000000,0.000000,0.000000,...,35.739316,22.094017,5.794872,3.445596,197.615385,0.663336,937.223816,2025,6,10
1030,2025-10-27,163.0,0.000000,0.000000,0.0,0.251282,0.000000,0.000000,0.000000,0.000000,...,35.887179,20.369230,6.764103,3.266129,220.733333,0.624237,869.557068,2025,0,10
1031,2025-10-28,76.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,36.000000,23.551723,12.448276,5.000000,6.586207,0.732618,886.694031,2025,1,10
1032,2025-10-29,38.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,35.022727,28.590910,3.863636,4.677419,51.613636,0.555750,917.601135,2025,2,10


In [21]:
ruta_principal = os.getcwd()
BuenasCarac_path = os.path.join(ruta_principal, "Files", "BuenaCaracteristicas.json")
with open(BuenasCarac_path, 'r') as f:
    #  Cargar el contenido del archivo JSON
    BC_json = json.load(f)
    
df_agregado= Get_goodCarac(df_agregado,BC_json)

In [22]:
df_agregado

,FECHA_CORRIDA,VENTABOL_DIARIA,PROP_HoraSalidaOrigenCorrida_0,PROP_HoraSalidaOrigenCorrida_1,PROP_HoraSalidaOrigenCorrida_10,PROP_HoraSalidaOrigenCorrida_11,PROP_HoraSalidaOrigenCorrida_12,PROP_HoraSalidaOrigenCorrida_13,PROP_HoraSalidaOrigenCorrida_14,PROP_HoraSalidaOrigenCorrida_15,...,DISPONIBILIDAD_TRAMO,PORCENT_PROMO,HORAS_ANTICIPACION,DIF_TARIF,VENTA,Anio,DiaSemana,Mes,Buen_Dia,Buen_Mes
0,2023-01-01,157.0,0.000000,0.000000,0.0,0.000000,0.000000,0.169492,0.000000,0.039548,...,4.525424,10.465117,4.451977,0.544464,714.251465,2023,6,1,1,1
1,2023-01-02,211.0,0.094118,0.113725,0.0,0.007843,0.133333,0.039216,0.113725,0.035294,...,6.690196,10.275229,12.298039,0.486919,696.876343,2023,0,1,1,1
2,2023-01-03,448.0,0.000000,0.053131,0.0,0.011385,0.000000,0.060721,0.041746,0.227704,...,4.969639,11.102942,16.571157,0.508980,697.675415,2023,1,1,0,1
3,2023-01-04,426.0,0.000000,0.075435,0.0,0.052224,0.000000,0.029014,0.069632,0.054159,...,6.959381,10.136987,14.059961,0.470546,665.716064,2023,2,1,0,1
4,2023-01-05,273.0,0.000000,0.064725,0.0,0.051780,0.038835,0.000000,0.100324,0.058252,...,17.572816,10.434783,13.779935,0.533888,670.463806,2023,3,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,2025-10-26,206.0,0.000000,0.000000,0.0,0.264957,0.000000,0.000000,0.000000,0.000000,...,5.794872,3.445596,197.615385,0.663336,937.223816,2025,6,10,1,0
1030,2025-10-27,163.0,0.000000,0.000000,0.0,0.251282,0.000000,0.000000,0.000000,0.000000,...,6.764103,3.266129,220.733333,0.624237,869.557068,2025,0,10,1,0
1031,2025-10-28,76.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,12.448276,5.000000,6.586207,0.732618,886.694031,2025,1,10,0,0
1032,2025-10-29,38.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.863636,4.677419,51.613636,0.555750,917.601135,2025,2,10,0,0


In [23]:
df_agregado.to_csv('Data4LSTM.csv')

In [24]:
columnas = [
    'PROP_HoraSalidaOrigenCorrida_0',
    'PROP_HoraSalidaOrigenCorrida_1',
    'PROP_HoraSalidaOrigenCorrida_10',
    'PROP_HoraSalidaOrigenCorrida_11',
    'PROP_HoraSalidaOrigenCorrida_12',
    'PROP_HoraSalidaOrigenCorrida_13',
    'PROP_HoraSalidaOrigenCorrida_14',
    'PROP_HoraSalidaOrigenCorrida_15',
    'PROP_HoraSalidaOrigenCorrida_16',
    'PROP_HoraSalidaOrigenCorrida_17',
    'PROP_HoraSalidaOrigenCorrida_18',
    'PROP_HoraSalidaOrigenCorrida_19',
    'PROP_HoraSalidaOrigenCorrida_2',
    'PROP_HoraSalidaOrigenCorrida_20',
    'PROP_HoraSalidaOrigenCorrida_21',
    'PROP_HoraSalidaOrigenCorrida_22',
    'PROP_HoraSalidaOrigenCorrida_23',
    'PROP_HoraSalidaOrigenCorrida_5',
    'PROP_HoraSalidaOrigenCorrida_6',
    'PROP_HoraSalidaOrigenCorrida_7',
    'PROP_HoraSalidaOrigenCorrida_8',
    'PROP_HoraSalidaOrigenCorrida_9',
    'PROP_PAGO_METODO_EFECTIVO',
    'PROP_PAGO_METODO_TARJETA',
    'PROP_TIPO_PASAJERO_AD',
    'PROP_TIPO_PASAJERO_ES',
    'PROP_TIPO_PASAJERO_IN',
    'PROP_TIPO_PASAJERO_MA',
    'PROP_TIPO_PASAJERO_NI',
    'PROP_TIPO_PASAJERO_OD',
    'PROP_TIPO_PASAJERO_PE',
    'PROP_TIPO_PASAJERO_PF',
    'PROP_CLASE_SERVICIO_DE LUJO',
    'PROP_CLASE_SERVICIO_DOS PISOS ETN',
    'PROP_CLASE_SERVICIO_DOS PISOS TURISTAR',
    'PROP_CLASE_SERVICIO_DTM DOS PISOS',
    'PROP_CLASE_SERVICIO_TURISTAR LUJO',
    'PROP_CLASE_SERVICIO_UN PISO DE LUJO',
    'PROP_CLASE_SERVICIO_UN PISO TURISTAR LUJO',
    'KMS_TRAMO',
    'CAPACIDAD_ASIENTOS_TRAMO',
    'OCUPACION_TRAMO',
    'DISPONIBILIDAD_TRAMO',
    'PORCENT_PROMO',
    'HORAS_ANTICIPACION',
    'DIF_TARIF',
    'VENTA',
    'Anio',
    'DiaSemana'
]